In [ ]:
from IPython.display import display, Math
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import scipy as scipy    

# Eigenvectors of the stiffness matrix in limit state problems
[Dominik Kern](https://orcid.org/0000-0002-1958-2982),
[Chiara Hergl](https://orcid.org/0000-0002-4016-9113),
[Thomas Nagel](https://orcid.org/0000-0001-8459-4616)
# Minimal Example of Beam Buckling with Axial-transversal Coupling
With this formulation we obtain a common FEM-like equation $\mathbf{C}({\mathbf{u}})\mathbf{u} = \mathbf{f}$.

![Euler buckling case 2](euler_buckling.svg)

Non-dimensional potential energy for coupled beam bending (transversal displacement $w$, axial displacement $u$) with axial force $F$ and stabilizing spring (stiffness $c$)    
$\Pi = \frac{1}{2}\int_0^1 w''(x)^2 + G w'^2 \mathrm{d}x +\frac{1}{2}cw(X)^2 + F u(1)$

with following relations to the dimensional variables 
- $G = \dfrac{A l^2}{I}$ squared slenderness ratio,
- $c = \dfrac{c_\mathrm{dim}l}{F_\mathrm{ref}}$,
- $F = \dfrac{F_\mathrm{dim}}{F_\mathrm{ref}}$,
with $F_\mathrm{ref} = \dfrac{EI}{l^2}$, note that first buckling load of this case is $F_\mathrm{crit} = \pi^2 F_\mathrm{ref}$.

Ritz' method with discretization $w(x)\approx \sum\limits_{k=1}^{K} \sin\bigl(k\pi x \bigr)$ and $u\approx \hat{u}x$

## Symbolic calculations

Apply Ritz discretization (1 axial and 2 transversal DoF) and evaluate first variation of potential energy

In [ ]:
x, X, k, c, F, W1, W2, dW1, dW2, U1, dU1, G  = sp.symbols('x X k c F W1 W2 dW1 dW2 U1 dU1 G')

In [ ]:
w = sp.sin(k*sp.pi*x) 

W = W1*w.subs(k,1) + W2*w.subs(k,2)
Wx = W.diff(x)
Wxx = Wx.diff(x)

dW = dW1*w.subs(k,1) + dW2*w.subs(k,2)
dWx = dW.diff(x)
dWxx = dWx.diff(x)

U = U1*x
Ux = U.diff(x)

dU = dU1*x
dUx = dU.diff(x)

dPi = sp.integrate(Wxx*dWxx + G*(Ux+(1/2)*Wx*Wx)*(dUx+Wx*dWx), (x,0,1) ) + c*W.subs(x,X)*dW.subs(x,X) + F*U.subs(x,1)
display(dPi)

Generalized forces

In [ ]:
Q1 = sp.simplify(dPi.diff(W1))
display(Math(f'Q_1 = {sp.latex(Q1)}'))
Q2 = sp.simplify(dPi.diff(W2))
display(Math(f'Q_2 = {sp.latex(Q2)}'))
Q3 = sp.simplify(dPi.diff(U1))
display(Math(f'Q_3 = {sp.latex(Q3)}'))

Stiffness matrix (symmetric, remember $\sin(2x)=2\sin(x)\cos(x)$)

In [ ]:
C11 = sp.expand(Q1).coeff(dW1, 1)
C12 = sp.expand(Q1).coeff(dW2, 1)
C13 = sp.expand(Q1).coeff(dU1, 1)

C21 = sp.expand(Q2).coeff(dW1, 1)
C22 = sp.expand(Q2).coeff(dW2, 1)
C23 = sp.expand(Q2).coeff(dU1, 1)

C31 = sp.expand(Q3).coeff(dW1, 1)
C32 = sp.expand(Q3).coeff(dW2, 1)
C33 = sp.expand(Q3).coeff(dU1, 1)

C = sp.Matrix(((C11, C12, C13), (C21, C22, C23), (C31, C32, C33)))
display(Math(f'C = {sp.latex(C)}'))

Stiffness matrix before buckling, i.e. without bending deformation ($w(x)=0$).

In [ ]:
Wzero = 0
Czero = C.subs([(W1, Wzero), (W2, Wzero)])
display(Math(f'C_0 = {sp.latex(Czero)}'))

Buckling occurs when the determinant of the stiffness matrix vanishes.

In [ ]:
char_eq = sp.simplify(sp.Determinant(C).doit())
buckling_state = sp.solve(char_eq, U1)
print(str(len(buckling_state)) + " buckling states (as many as transversal DoF)")
U_1 = sp.simplify(buckling_state[0])
U_2 = sp.simplify(buckling_state[1])
display(Math(f'U_{1} = {sp.latex(U_1)}'))
display(Math(f'U_{2} = {sp.latex(U_2)}'))

Uzero_1 = U_1.subs([(W1, Wzero), (W2, Wzero)])
Uzero_2 = U_2.subs([(W1, Wzero), (W2, Wzero)])

## Numerical evaluations

common value for the slenderness ratio and a wide range of spring stiffness

In [ ]:
N = 100
c_grid = np.linspace(0, 300,N)   
Gnum = 100**2   
F1_sym = np.zeros((N,1))
F2_sym = np.zeros((N,1))
F1_asym = np.zeros((N,1))
F2_asym = np.zeros((N,1))
ev1 = np.zeros((N,3))
ev2 = np.zeros((N,3))
sev = np.zeros((N,3))

### stabilizing spring in the middle ($X=\frac{1}{2}$) _symmetric case_

In [ ]:
Xsym = 0.5

Firstly, solve eigenvalue problem manually for verification

In [ ]:
#for k,cnum in enumerate(c_grid):
#    Usym_1 = Uzero_1.subs([(X,Xsym), (c,cnum,), (G,Gnum)])
#    Usym_2 = Uzero_2.subs([(X,Xsym), (c,cnum,), (G,Gnum)])
#
#    Czero_sym = Czero.subs([(X,Xsym),(c,cnum),(G,Gnum)])
#    F1_sym[k] = np.abs(Usym_1)*Czero_sym.row(2).col(2)[0]   # TODO correct here, but matrix/vector is general
#    F2_sym[k] = np.abs(Usym_2)*Czero_sym.row(2).col(2)[0]
#    display()
    
    # solve for Eigenvectors
    #Czero_num1 = Czero_sym.subs(U1, Usym_1)
    #Czero_num2 = Czero_sym.subs(U1, Usym_2)

Secondly, set up generalized eigenvalue problem $\mathbf{A}\mathbf{w} = x\mathbf{B}\mathbf{w}$ and compare with standard eigenvectors at $U=U_\mathrm{crit}$ from above


In [ ]:
for k,cnum in enumerate(c_grid):
    Czero_sym = Czero.subs([(X,Xsym),(c,cnum),(G,Gnum)])

    # C = A + B*U1    
    Azero_sym = sp.Matrix([[(Czero_sym[i,j].coeff(U1,0).evalf()) for i in [0,1,2]] for j in [0,1,2]])
    Bzero_sym = sp.Matrix([[(Czero_sym[i,j].coeff(U1,1).evalf()) for i in [0,1,2]] for j in [0,1,2]])
    Azero_num = np.array(Azero_sym).astype(np.float64)
    Bzero_num = np.array(Bzero_sym).astype(np.float64)

    eigvals, eigvecs = scipy.linalg.eig(Azero_num, Bzero_num)
    C33num = Czero_sym.row(2).col(2)[0]
    # sort by eigenvalues
    if eigvals[0] < eigvals[1]:
        F1_sym[k] = eigvals[0].real*C33num
        F2_sym[k] = eigvals[1].real*C33num
    else:
        F1_sym[k] = eigvals[1].real*C33num
        F2_sym[k] = eigvals[0].real*C33num

Plot buckling load for a range of spring stiffnesses

In [ ]:
plt.plot(c_grid,F1_sym,'b', c_grid,F2_sym,'r')
plt.xlabel("stiffness");
plt.ylabel("buckling load");
plt.legend(["F_1", "F_2"]);

Note that the horizontal line, independent on color, corresponds to the second mode shape and the sloped line to the first mode shape. The Physics behind this plot are that, the spring only stabilizes against first mode buckling and is ineffective for the second mode (node point). There is one value of the spring, when both buckling loads are equal.

### stabilizing spring somewhere ($X=\frac{3}{4}$) _asymmetric case_

In [ ]:
Xasym = 0.55

In [ ]:
for k,cnum in enumerate(c_grid):
    Czero_asym = Czero.subs([(X,Xasym),(c,cnum),(G,Gnum)])
    
    Azero_asym = sp.Matrix([[(Czero_asym[i,j].coeff(U1,0).evalf()) for i in [0,1,2]] for j in [0,1,2]])
    Bzero_asym = sp.Matrix([[(Czero_asym[i,j].coeff(U1,1).evalf()) for i in [0,1,2]] for j in [0,1,2]])

    Azero_anum = np.array(Azero_asym).astype(np.float64)
    Bzero_anum = np.array(Bzero_asym).astype(np.float64)

    eigvals, eigvecs = scipy.linalg.eig(Azero_anum, Bzero_anum)

    C33num = Czero_asym.row(2).col(2)[0]
    # sort by eigenvalues and normalize eigenvectors
    if eigvals[0] < eigvals[1]:
        F1_asym[k] = eigvals[0].real*C33num
        ev1_abs = np.linalg.norm(eigvecs[0])
        ev1[k,0] = eigvecs[0][0]/ev1_abs
        ev1[k,1] = eigvecs[0][1]/ev1_abs
        ev1[k,2] = eigvecs[0][2]/ev1_abs
        F2_asym[k] = eigvals[1].real*C33num
        ev2_abs = np.linalg.norm(eigvecs[1])
        ev2[k,0] = eigvecs[1][0]/ev2_abs
        ev2[k,1] = eigvecs[1][1]/ev2_abs
        ev2[k,2] = eigvecs[1][2]/ev2_abs
    else:
        F1_asym[k] = eigvals[1].real*C33num
        ev1_abs = np.linalg.norm(eigvecs[1])
        ev1[k,0] = eigvecs[1][0]/ev1_abs
        ev1[k,1] = eigvecs[1][1]/ev1_abs
        ev1[k,2] = eigvecs[1][2]/ev1_abs
        F2_asym[k] = eigvals[0].real*C33num
        ev2_abs = np.linalg.norm(eigvecs[0])
        ev2[k,0] = eigvecs[0][0]/ev2_abs
        ev2[k,1] = eigvecs[0][1]/ev2_abs
        ev2[k,2] = eigvecs[0][2]/ev2_abs

    # solve standard eigenvalue problem at lower buckling load
    U1lower = min(eigvals[0], eigvals[1])
    Czero_anum = np.matrix(Czero_asym.subs(U1, U1lower).evalf()).astype(np.float64)
    standard_eigvals, standard_eigvecs = scipy.linalg.eig(Czero_anum)
    sev_abs = np.linalg.norm(eigvecs[0])
    sev[k,0] = standard_eigvecs[0][0]/sev_abs
    sev[k,1] = standard_eigvecs[0][1]/sev_abs
    sev[k,2] = standard_eigvecs[0][2]/sev_abs

In [ ]:
plt.plot(c_grid,F1_asym,'b', c_grid,F2_asym,'r')
plt.xlabel("stiffness");
plt.ylabel("buckling load");
plt.legend(["F_1", "F_2"]);

There is no common value of two buckling modes for any spring stiffness.

#### Eigenvectors

In [ ]:
plt.plot(c_grid, np.abs(ev1[:,0]),'b', c_grid, np.abs(ev1[:,1]),'b--', c_grid, np.abs(ev1[:,2]),'b:');
plt.plot(c_grid, np.abs(sev[:,0]),'k', c_grid, np.abs(sev[:,1]),'k--', c_grid, np.abs(sev[:,2]),'k:');
plt.xlabel('$c$');
plt.legend(['$|w_1| (gen.EVP)$', '$|w_2| (gen.EVP)$', '$|u_1|$ (gen.EVP)', '$|w_1| (sta.EVP)$', '$|w_2| (sta.EVP)$', '$|u_1|$ (sta.EVP)']);

For the lower buckling load there is a transition from the first mode shape to the second, relation of generalized EVP to standard EVP is still mysterious **TODO**

In [ ]:
plt.plot(c_grid, np.abs(ev2[:,0]),'r', c_grid, np.abs(ev2[:,1]),'r--', c_grid, np.abs(ev2[:,2]),'r:');
plt.xlabel('$c$');
plt.legend(['$|w_1|$', '$|w_2|$', '$|u_1|$']);

For the higher buckling load it is the opposite transition than for the lower buckling load.

#### Plots of the buckling shapes for different values of the spring stiffness

In [ ]:
x_grid = np.linspace(0, 1) 
index_c = [1, 48, 99] 
for i_c in index_c:
    w_buckling_lower_load  = lambda x: ev1[i_c,0] * np.sin(np.pi*x) + ev1[i_c,1] * np.sin(2*np.pi*x) 
    w_buckling_higher_load = lambda x: ev2[i_c,0] * np.sin(np.pi*x) + ev2[i_c,1] * np.sin(2*np.pi*x) 
    plt.plot(x_grid, w_buckling_lower_load(x_grid),'b')
    plt.plot(x_grid, w_buckling_higher_load(x_grid),'r')
    plt.xlabel('x')
    plt.ylabel('w')
    plt.legend(['F_1', 'F_2'])
    plt.title('c=' + str(c_grid[i_c]))
    plt.xticks([0, 0.2, 0.4, 0.6, 0.8, 1, Xasym], ['0', '0.2', '0.4', '0.6', '0.8', '1', 'X'])
    plt.show(); 
    display(Math(f'w_l(X) = {sp.latex(w_buckling_lower_load(Xasym))}')) 
    display(Math(f'w_h(X) = {sp.latex(w_buckling_higher_load(Xasym))}'))